In [1]:
from main import environment
import random,sys,gc,warnings
from IPython.display import clear_output

import gymnasium 
from gymnasium.vector import SyncVectorEnv

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
from torch.utils.tensorboard import SummaryWriter

from tqdm import tqdm

In [ ]:
torch.autograd.set_detect_anomaly(True)
warnings.filterwarnings("ignore")

# hypers
batchSize = 10
lr = 1e-4
num_env = 3



    #
    
env = gymnasium.make("sudoku")
env = SyncVectorEnv([lambda : env]*num_env)

In [ ]:
class collector:
    def __init__(self):
        pass
        
    def rollout(self):
        pass

    def sample(self):
        pass

    def clear_memory():
        pass

In [ ]:
class mask: # altering softmax output so x and y = {0,8} and value = {1,9}
  def __init__(self):
    self.newValue = -float("inf")

  def apply(self,tensor : torch.FloatTensor):
    self.mask = torch.zeros_like(tensor,dtype=torch.bool)
    self.mask[0,-1] = True
    self.mask[1,-1] = True
    self.mask[-1,0] = True
    tensor = tensor.masked_fill(mask=self.mask,value=self.newValue)
    return tensor

class network(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.LazyConv2d(1,(1,1))
    self.conv2 = nn.LazyConv2d(1,(1,1))
    self.conv3 = nn.LazyConv2d(1,(1,1))

    self.linear1 = nn.LazyLinear(9)
    self.linear2 = nn.LazyLinear(9)
    self.linear3 = nn.LazyLinear(9)

    self.policy_mask = mask()
    self.policy_head = nn.LazyLinear(30)
    self.value_head = nn.LazyLinear(1)
    
    self.optimizer = torch.optim.Adam(self.parameters(),lr=lr)
    
  def forward(self,x):
    x = F.relu(self.conv1(x))
    x = self.conv2(x)
    x = F.relu(self.conv3(x))
    x = torch.flatten(x,start_dim=1)
    x = self.linear1(x)
    x = F.relu(self.linear2(x))
    x = self.linear3(x)
    distibution = F.relu(self.policy_head(x)).reshape(3,10)
    distibution = self.policy_mask.apply(distibution)
    value = self.value_head(x)
    return F.softmax(distibution,-1),value

d = network()
d.forward(torch.rand((1,9,9),dtype=torch.float))
clear_output()